In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [33]:
class ModelEstimatorHelper:
    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=1, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs, 
                              verbose=verbose, scoring=scoring, refit=refit)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': np.min(scores),
                 'max_score': np.max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series(dict(params.items() + d.items()))

        rows = [row(k, gsc.cv_validation_scores, gsc.parameters) 
                     for k in self.keys
                     for gsc in self.grid_searches[k].grid_scores_]
        df = pd.concat(rows, axis=1).T.sort([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [29]:
from sklearn import datasets

iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target

In [35]:
from sklearn.ensemble import (ExtraTreesClassifier, RandomForestClassifier, 
                              AdaBoostClassifier, GradientBoostingClassifier)
from sklearn.svm import SVC

models1 = { 
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'SVC': SVC()
}

params1 = { 
    'ExtraTreesClassifier': { 'n_estimators': [16, 32, 50] },
    'RandomForestClassifier': { 'n_estimators': [16, 32, 50] },
    'AdaBoostClassifier':  { 'n_estimators': [16, 32] },
    'GradientBoostingClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] },
    'SVC': [
        {'kernel': ['linear'], 'C': [1, 10]},
        {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.001, 0.0001]},
    ]
}

In [31]:
helper1 = ModelEstimatorHelper(models1, params1)
helper1.fit(X_iris, y_iris, n_jobs=-1)

Running GridSearchCV for SVC.
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Running GridSearchCV for AdaBoostClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.1s finished


Running GridSearchCV for GradientBoostingClassifier.
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.2s remaining:    0.3s


Running GridSearchCV for ExtraTreesClassifier.
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished


Running GridSearchCV for RandomForestClassifier.
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.9s finished


In [38]:
helper1.score_summary()

/home/sekhar/miniconda2/envs/ipy_env/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/sekhar/miniconda2/envs/ipy_env/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/sekhar/miniconda2/envs/ipy_env/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/sekhar/miniconda2/envs/ipy_env/lib/pytho

,estimator,min_score,mean_score,max_score,std_score,C,gamma,kernel,learning_rate,n_estimators
0,SVC,0.960784,0.979984,1,0.0160202,1,NaN,linear,NaN,NaN
1,SVC,0.921569,0.973856,1,0.0369729,10,NaN,linear,NaN,NaN
8,GradientBoostingClassifier,0.921569,0.96732,1,0.0333269,NaN,NaN,NaN,0.8,16
13,ExtraTreesClassifier,0.921569,0.96732,1,0.0333269,NaN,NaN,NaN,NaN,32
11,GradientBoostingClassifier,0.921569,0.96732,1,0.0333269,NaN,NaN,NaN,1,32
10,GradientBoostingClassifier,0.921569,0.96732,1,0.0333269,NaN,NaN,NaN,1,16
9,GradientBoostingClassifier,0.921569,0.96732,1,0.0333269,NaN,NaN,NaN,0.8,32
12,ExtraTreesClassifier,0.941176,0.966912,0.980392,0.0182045,NaN,NaN,NaN,NaN,16
15,RandomForestClassifier,0.941176,0.966912,0.980392,0.0182045,NaN,NaN,NaN,NaN,16
14,ExtraTreesClassifier,0.901961,0.960784,1,0.0423578,NaN,NaN,NaN,NaN,50


In [45]:
diabetes = datasets.load_diabetes()
X_db = diabetes.data
Y_db = diabetes.target

In [47]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

models2 = { 
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso()
}

params2 = { 
    'LinearRegression': { },
    'Ridge': { 'alpha': [0.1, 1.0] },
    'Lasso': { 'alpha': [0.1, 1.0] }
}

In [50]:
helper2 = ModelEstimatorHelper(models2, params2)
helper2.fit(X_db, Y_db, n_jobs=-1)

Running GridSearchCV for Ridge.
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Running GridSearchCV for LinearRegression.
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.3s finished


Running GridSearchCV for Lasso.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.1s finished


In [51]:
helper2.score_summary()

/home/sekhar/miniconda2/envs/ipy_env/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/sekhar/miniconda2/envs/ipy_env/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/sekhar/miniconda2/envs/ipy_env/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/sekhar/miniconda2/envs/ipy_env/lib/pytho

,estimator,min_score,mean_score,max_score,std_score,alpha
2,LinearRegression,0.469306,0.488703,0.509553,0.0164628,NaN
0,Ridge,0.458568,0.488607,0.512092,0.0223373,0.1
3,Lasso,0.455376,0.486668,0.507581,0.0225404,0.1
1,Ridge,0.370016,0.409427,0.439819,0.0292018,1
4,Lasso,0.34543,0.3538,0.368847,0.0106624,1
